In [1]:
###### Config #####
import sys, os, platform
if os.path.isdir("ds-assets"):
  !cd ds-assets && git pull
else:
  !git clone https://github.com/lutzhamel/ds-assets.git
colab = True if 'google.colab' in os.sys.modules else False
system = platform.system() # "Windows", "Linux", "Darwin"
home = "ds-assets/assets/"
sys.path.append(home)  

Already up to date.


In [2]:
# notebook level imports
import dsutils  # DBCredentials, execute_query

![](https://www.quest.com/images/og/OGImage-RelationalDatabase_166844.png)

# Relational Databases

* The data you need will often live in **databases**, systems designed for efficiently storing and querying data.

* The bulk of these are relational databases, such as Oracle, MySQL, and SQL Server.  These are also called **Relational Database Management Systems** ([RDBMS](https://en.wikipedia.org/wiki/Relational_database_management_system)).

* These systems store data in tables and are typically queried using the **Structured Query Language** ([SQL](https://en.wikipedia.org/wiki/SQL)), a declarative language for manipulating data.



## What is a Relational Database?

* A relational database is a **collection of tables**.
   * A **table** is simply a collection of rows and columns, very **similar to Pandas DataFrames**.
   * A database typically contains **multiple tables**.
* Each table typically has columns called the **primary and/or foreign keys**.
   * These special columns allow the user to pose queries across multiple different tables at the same time - to perform **joins across tables**.
   * A **primary key** is a column that holds a **unique value for each row** in the table.  This is used by the db engine to optimize queries against the table.
   * A **foreign key** is a column where each value **points to the primary key of another tabl**e.  So you can think of a foreign key as pointer from one table to another.
* Tables together with primary/foreign key relationships are called the **schema of a database**.
* SQL is used to query the data in a relational database.  
* Data returned from an SQL query is returned as a table -- the **result table**

These databases are called relational because each table defines a [mathematical relation](https://www.vertabelo.com/blog/why-are-relational-databases-relational/)!



## SQL Queries and Result Tables

![alt](https://www.w3resource.com/w3r_images/sql-works-with-rdbms.gif)

# SQL

SQL (Structured Query Language) is a domain-specific language used for programming and managing data held in a relational database management system.

SQL was one of the first languages for Edgar F. Codd's relational model, as described in his influential 1970 paper, "[A Relational Model of Data for Large Shared Data Banks](https://sfu-db.github.io/dbsystems/Papers/p377-codd.pdf)."

Here is a nice [SQL tutorial](https://www.w3schools.com/sql).


## What Can SQL do?
* SQL can execute queries against a database
* SQL can retrieve data from a database
* SQL can insert records in a database
* SQL can update records in a database
* SQL can delete records from a database
* SQL can create new databases
* SQL can create new tables in a database
* SQL can create stored procedures in a database
* SQL can create views in a database
* SQL can set permissions on tables, procedures, and views




## SQL is actually made up of a couple of sub-languages:

* DDL: Data Definition Language, e.g. ‘create’ a table or database
* DML: Data Manipulation Language, e.g. insert or delete a row in a table
* TCL: Transaction Control Language, e.g. commit or rollback database changes
* DCL: Data Control Language, e.g. grant access permissions
* **DQL: Data Query Language**, e.g. retrieve records from one or more table

**Note:** Only a small part of SQL actually has to do with information retrieval/querying

## Data Retrieval with `SELECT`

From our perspective, the most important feature in SQL is the **SELECT** statement that allows us to extract data from the DB tables:
```
SELECT * FROM Customers;               -- get entire contents of table Customers
SELECT * FROM Customers LIMIT 2;       -- get the first two rows
SELECT CustomerID,CustomerName FROM Customers;    
                                       -- get columns CustomerID, CustomerName of table Customers
SELECT CustomerName,City FROM Customers WHERE CustomerID = 3;  
                                       -- get data subject to some conditions
```

Take a peek at the W3Schools [reference manual for SQL](https://www.w3schools.com/sql/).

# SQL and Python

We will use the [PyMySQL](https://pymysql.readthedocs.io/en/latest/) package together with  [Pandas DataFrames' ability to query databases using SQL](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html) to connect to a [MySQL server](https://www.mysql.com) and issue SQL commands.

We will use the 'world' database from the [MySQL website](https://dev.mysql.com/doc/world-setup/en).

In [3]:
# our database credentials
credentials = dsutils.DBCredentials(
   host = 'testdb.cwy05wfzuxbv.us-east-1.rds.amazonaws.com',
   userdb = 'world',
   user = 'csc310',
   password = 'csc310$is$fun')

The **schema** of the world database looks as follows:

<img src="https://static.packt-cdn.com/products/9781788390415/graphics/cac1f609-1c45-46d7-b066-d9481ceddf18.png">

The database consists of three tables with the central table being the 'country' table.  The other two tables are related to the 'country' table via the 'CountryCode' columns that are set up as **foreign keys**.

## Basic Queries

### Mimicking the Pandas 'head' Function

Here is a program that queries 10 rows from the table 'city'.  Notice, that the results are returned as a Pandas dataframe and therefore, once we have the results, we can apply standard Pandas things to the results if we so chose (which will do later on).

In [4]:

# return the first 10 rows from the city table
# using a SQL query string
sql_string = \
'''
SELECT 
   *        -- everything
FROM 
   city     -- table city
LIMIT 
   10       -- limit to first 10 rows
'''

In [5]:
dsutils.execute_query(credentials, sql_string)

,ID,Name,CountryCode,District,Population
0,1,Kabul,AFG,Kabol,1780000
1,2,Qandahar,AFG,Qandahar,237500
2,3,Herat,AFG,Herat,186800
3,4,Mazar-e-Sharif,AFG,Balkh,127800
4,5,Amsterdam,NLD,Noord-Holland,731200
5,6,Rotterdam,NLD,Zuid-Holland,593321
6,7,Haag,NLD,Zuid-Holland,440900
7,8,Utrecht,NLD,Utrecht,234323
8,9,Eindhoven,NLD,Noord-Brabant,201843
9,10,Tilburg,NLD,Noord-Brabant,193238


### Being more specific in what we are asking for!
Here is another program that queries the 'city' table for cities where the population is greater than 5,000,000
and displays the columns 'name', 'countrycode', and 'population'. It displays the results in descending order.

In [6]:

# show the name, countrycode, and population of cities 
# where the population is greater than 5,000,000,
# ordered by population in descending order
sql_string = \
'''
SELECT
    -- limit the columns to show
    name,
    countrycode,
    population
FROM
    city
WHERE
    population > 5000000 -- only cities with population > 5 million
ORDER
    BY population DESC  -- sort by population in descending order
'''

In [7]:
dsutils.execute_query(credentials, sql_string)


,name,countrycode,population
0,Mumbai (Bombay),IND,10500000
1,Seoul,KOR,9981619
2,São Paulo,BRA,9968485
3,Shanghai,CHN,9696300
4,Jakarta,IDN,9604900
5,Karachi,PAK,9269265
6,Istanbul,TUR,8787958
7,Ciudad de México,MEX,8591309
8,Moscow,RUS,8389200
9,New York,USA,8008278


### Query on how many distinct languages there are in the DB


In [8]:
# count how many distinct, official languages are in the DB
sql_string = \
'''
SELECT
   COUNT(DISTINCT language)  -- count distinct languages
FROM
   countrylanguage
WHERE
  isofficial = 'T'; -- only count official languages
'''

* Here we query all the languages that are considered official languages. 
* The 'distinct' keyword prevents
   a language to appear multiple times in the result table
* The 'count' function counts the number of distinct languages returned.

Noteworthy here is the fact that we can **execute functions like the 'count' function right on the DB server** without having to first export the data to our local machine!

In [9]:
dsutils.execute_query(credentials, sql_string)

,COUNT(DISTINCT language)
0,103


### Query DB Meta Information

We can use SQL to access meta-information about the data and the db.

In [10]:
data = dsutils.execute_query(credentials, 'SHOW TABLES')
data

,Tables_in_world
0,city
1,country
2,countrylanguage


In [11]:
# format the output nicely
print(f"There are {data.shape[0]} tables in the {credentials.userdb} database")
print("The tables are: " + ", ".join(list(data.iloc[:,0])))

There are 3 tables in the world database
The tables are: city, country, countrylanguage


We can look at columns of a table together with their meta-information.

In [12]:
data = dsutils.execute_query(credentials, 'SHOW COLUMNS FROM country')
data

,Field,Type,Null,Key,Default,Extra
0,Code,char(3),NO,PRI,,
1,Name,char(52),NO,,,
2,Continent,"enum('Asia','Europe','North America','Africa',...",NO,,Asia,
3,Region,char(26),NO,,,
4,SurfaceArea,"float(10,2)",NO,,0.00,
5,IndepYear,smallint(6),YES,,None,
6,Population,int(11),NO,,0,
7,LifeExpectancy,"float(3,1)",YES,,None,
8,GNP,"float(10,2)",YES,,None,
9,GNPOld,"float(10,2)",YES,,None,


In [13]:
# only show columns of interest to us in the order we want
data[['Key','Field','Type']]

,Key,Field,Type
0,PRI,Code,char(3)
1,,Name,char(52)
2,,Continent,"enum('Asia','Europe','North America','Africa',..."
3,,Region,char(26)
4,,SurfaceArea,"float(10,2)"
5,,IndepYear,smallint(6)
6,,Population,int(11)
7,,LifeExpectancy,"float(3,1)"
8,,GNP,"float(10,2)"
9,,GNPOld,"float(10,2)"


Try the above queries for tables `city` and `countrylanguage`.

## Joins

A [SQL join](https://en.wikipedia.org/wiki/Join_(SQL)) combines columns from different tables in a relational database by using values common to each.  The most common join is the `INNER JOIN` which constructs a return set with the rows from each table for which the join condition is true.



This is where the foreign keys come in handy -- recall that foreign keys are like pointers from one table to another and are used in a join to create the joined result set,

<img src="https://static.packt-cdn.com/products/9781788390415/graphics/cac1f609-1c45-46d7-b066-d9481ceddf18.png">



The SQL select statement allows you to construct a result set made up of columns from different tables.  The basic syntax of the command for this is as follows,

```
SELECT
  <table qualifier>.<column name>,
  <table qualifier>.<column name>,
    ...
  <table qualifier>.<column name>   <<<-- NO COMMA ON THE LAST ONE!
FROM
  <reference table name>
JOIN
  <join table 1>
ON
  <join condition 1>
    ...
JOIN
  <joint table n>
ON
  <join condition n>;
```

The `<table qualifier>` in the result set construction is either the reference table name or one of the join table names.  The `<column name>` refers to a column in the qualified table.

Let's try some joins in our world database.

We want to print out the top 10 city names together with their country names and their populations.  All we need to do is select appropriate columns from the join result table.

In [14]:
# print out the top 10 city names with country names and populations.

# Notice the table qualifiers on the column names
# also notice that we can rename columns in the query to more
# more convenient names

sql_string = \
'''
SELECT
    city.name AS City,
    country.name AS Country,
    city.population AS CityPopulation
FROM
    city
JOIN         -- join city and country tables
    country
ON
    city.countrycode = country.code  -- foreign/primary key
ORDER
    BY city.population DESC
LIMIT
    10
'''


In [15]:
dsutils.execute_query(credentials, sql_string)

,City,Country,CityPopulation
0,Mumbai (Bombay),India,10500000
1,Seoul,South Korea,9981619
2,São Paulo,Brazil,9968485
3,Shanghai,China,9696300
4,Jakarta,Indonesia,9604900
5,Karachi,Pakistan,9269265
6,Istanbul,Turkey,8787958
7,Ciudad de México,Mexico,8591309
8,Moscow,Russian Federation,8389200
9,New York,United States,8008278


Let's try another one: let's query each city with its population and its country's population.  We will also compute and print out what fraction of the overall country population the city population represents.

In [16]:
# let's query each city with its population and its country's population.
sql_string = \
'''
SELECT
    country.name AS Country,
    city.name AS City,
    city.population AS CityPop,
    country.population AS CountryPop,
    city.population / country.population AS Fraction
FROM
    city
JOIN
    country
ON
    city.countrycode = country.code
ORDER
    BY Fraction DESC
LIMIT
    10
'''

In [17]:
dsutils.execute_query(credentials, sql_string)

,Country,City,CityPop,CountryPop,Fraction
0,Singapore,Singapore,4017733,3567000,1.1264
1,Gibraltar,Gibraltar,27025,25000,1.0810
2,Macao,Macao,437500,473000,0.9249
3,Pitcairn,Adamstown,42,50,0.8400
4,Cocos (Keeling) Islands,Bantam,503,600,0.8383
5,Saint Pierre and Miquelon,Saint-Pierre,5808,7000,0.8297
6,Falkland Islands,Stanley,1636,2000,0.8180
7,Palau,Koror,12000,19000,0.6316
8,Djibouti,Djibouti,383000,638000,0.6003
9,Cook Islands,Avarua,11900,20000,0.5950


One more! Print out how many cities are recorded for the USA, their average population, total population of US.

In [18]:
# print out number of US cities in the DB together with their average population
sql_string = \
'''
SELECT
    COUNT(city.name) as number,
    AVG(city.population) as avg_pop,
    country.population as population
FROM
    city
JOIN
    country
ON
    city.countrycode = country.code
WHERE
    country.code = 'USA';
'''


In [19]:
dsutils.execute_query(credentials, sql_string)

,number,avg_pop,population
0,274,286955.3796,278357000
